In [1]:
import pandas as pd
import os

datasets = '../data/dataset'
files_list = os.listdir(datasets)

data = pd.DataFrame()

for file in files_list:
    filepath = os.path.join(datasets, file)
    df = pd.read_csv(filepath)
    data = pd.concat([data, df], ignore_index=True)

data.insert(0,'id','')
data['id'] = data.audio_path.str[-10:]
data.head(5)

,id,audio_path,transcript,summary
0,000000.mp3,../data/audio\000000.mp3,"\n\tON the north-east coast of Scotland, in th...",\n \n\tThe history of the family of the...
1,000001.mp3,../data/audio\000001.mp3,\n\tALLEYN was no where to be found. The Earl ...,\n \n\tThere is an attack and an impend...
2,000002.mp3,../data/audio\000002.mp3,\n\tTHE Count was walking on the ramparts of t...,\n \n\tMalcolm reveals an important sec...
3,000003.mp3,../data/audio\000003.mp3,\n\tMEANWHILE the Earl remained a solitary pri...,\n \n\tMatilda falls into despair over ...
4,000004.mp3,../data/audio\000004.mp3,"\n\tMARY, in the mean time, suffered all the t...",\n \n\tIdentities are revealed and the ...


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [18]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, TaskType
from datasets import Dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device.type == "cuda":
    torch.cuda.empty_cache()

model_name = "mistralai/Mistral-7B-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

mistral = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 

In [14]:
dataset = Dataset.from_pandas(data)

def preprocess_function(examples):
    inputs = examples['transcript']
    targets = examples['summary']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

split_dataset = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

Map:   0%|          | 0/390 [00:00<?, ? examples/s]

c:\Users\guyet\anaconda3\envs\deepl\Lib\site-packages\transformers\tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/390 [00:00<?, ? examples/s]

In [9]:
peft_config = LoraConfig(
        r=16,
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
    )

mistral_lora = get_peft_model(mistral, peft_config)

In [15]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_dir="./logs",
    evaluation_strategy="epoch",
)

trainer = Trainer(
    model=mistral_lora,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator
)

trainer.train()

c:\Users\guyet\anaconda3\envs\deepl\Lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/528 [00:00<?, ?it/s]

ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [ ]:
model_path = "./peft-mistral-summary"

mistral_lora.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)